In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from mobcom.models.neural_network import NeuralNetwork
from mobcom.environment import Optimizer
from mobcom.io.data_provider import DataProvider

In [2]:
#train_data, test_data = DataProvider.select("default")
test = DataProvider.select("default")

ValueError: could not determine the shape of object type 'DataFrame'

In [12]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [13]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=Optimizer.LEARNING_RATE)

In [15]:
def train(dataloader: DataLoader, model, loss_fn: nn.CrossEntropyLoss, optimizer: torch.optim.SGD):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}   [{current:>5d}/{size:>5d}]")


In [16]:
def test(dataloader: DataLoader, model, loss_fn:nn.CrossEntropyLoss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg. loss: {test_loss:>8f} ")
    

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------------
loss: 2.310673   [   64/60000]
loss: 2.290894   [ 6464/60000]
loss: 2.270659   [12864/60000]
loss: 2.263996   [19264/60000]
loss: 2.249777   [25664/60000]
loss: 2.221135   [32064/60000]
loss: 2.229175   [38464/60000]
loss: 2.193488   [44864/60000]
loss: 2.191582   [51264/60000]
loss: 2.160387   [57664/60000]
Test Error: 
 Accuracy: 41.8%, Avg. loss: 2.155057 
Epoch 2
------------------------------------
loss: 2.167244   [   64/60000]
loss: 2.153565   [ 6464/60000]
loss: 2.098600   [12864/60000]
loss: 2.113091   [19264/60000]
loss: 2.058724   [25664/60000]
loss: 2.001342   [32064/60000]
loss: 2.028290   [38464/60000]
loss: 1.950722   [44864/60000]
loss: 1.952599   [51264/60000]
loss: 1.884051   [57664/60000]
Test Error: 
 Accuracy: 56.6%, Avg. loss: 1.885481 
Epoch 3
------------------------------------
loss: 1.914731   [   64/60000]
loss: 1.885630   [ 6464/60000]
loss: 1.778922   [12864/60000]
loss: 1.817254   [19264/60000]
loss: 1.699481   